In [72]:
import pyspark
from pyspark import SparkContext, SparkConf, SQLContext
from pyspark.sql import SparkSession

# from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName("ML_Model").getOrCreate()
# spark.conf.set("spark.executor.memory", "8g")
# spark.conf.set("spark.executor.cores", "4")
# spark.conf.set("spark.driver.cores", "4")
# conf = SparkConf().set("spark.jars", "/Users/sz904/Downloads/postgresql-42.2.18.jar")
# sc = SparkContext( conf=conf)



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/26 21:49:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [73]:
from pyspark.ml import Pipeline,Transformer
from pyspark.ml.feature import Imputer,StandardScaler,StringIndexer,OneHotEncoder, VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.sql.functions import *
from pyspark.sql.types import *
import numpy as np

In [100]:
col_names = ['tcp_flags',
 'tcp_time_delta',
 'tcp_len',
 'mqtt_conack_flags',
 'mqtt_conack_flags_reserved',
 'mqtt_conack_flags_sp',
 'mqtt_conack_val',
 'mqtt_conflag_cleansess',
 'mqtt_conflag_passwd',
 'mqtt_conflag_qos',
 'mqtt_conflag_reserved',
 'mqtt_conflag_retain',
 'mqtt_conflag_uname',
 'mqtt_conflag_willflag',
 'mqtt_conflags',
 'mqtt_dupflag',
 'mqtt_hdrflags',
 'mqtt_kalive',
 'mqtt_len',
 'mqtt_msg',
 'mqtt_msgid',
 'mqtt_msgtype',
 'mqtt_proto_len',
 'mqtt_protoname',
 'mqtt_qos',
 'mqtt_retain',
 'mqtt_sub_qos',
 'mqtt_suback_qos',
 'mqtt_ver',
 'mqtt_willmsg',
 'mqtt_willmsg_len',
 'mqtt_willtopic',
 'mqtt_willtopic_len',
 'target',
 'dataset',
 'tcp_flags_decimal',
 'mqtt_conflags_decimal',
 'mqtt_hdrflags_decimal']
train_path = "/Users/sz904/Desktop/df_train_pandas.csv"
test_path = "/Users/sz904/Desktop/df_test_pandas.csv"


In [101]:

df_train_updated = spark.read.csv(train_path, header=True, inferSchema= True).toDF(*col_names)
df_test_updated = spark.read.csv(test_path, header=True, inferSchema= True).toDF(*col_names)



In [106]:
#  "dataset" is removed from the cols, since it is only a indicator for 
#  for the train/test
df_train_updated = df_train_updated.drop("tcp_flags")
df_train_updated = df_train_updated.drop("mqtt_conflags")
df_train_updated = df_train_updated.drop("mqtt_hdrflags")
df_train_updated = df_train_updated.drop("dataset")

df_test_updated = df_test_updated.drop("tcp_flags")
df_test_updated = df_test_updated.drop("mqtt_conflags")
df_test_updated = df_test_updated.drop("mqtt_hdrflags")
df_test_updated = df_test_updated.drop("dataset")


df_train_updated = df_train_updated.drop("mqtt_msg")
df_test_updated = df_test_updated.drop("mqtt_msg")

In [107]:
Col_names = df_train_updated.columns
'''
 "dataset" is removed from the cols, since it is only a indicator for 
 for the train/test

corelated_cols_to_remove are dropped since after looking into them, 
values are all zeros in train dataset. Since the model can't learn
anything from it, and thus they should be removed. 
'''
nominal_cols = ["mqtt_msg"]

corelated_cols_to_remove = ["mqtt_conack_flags",
                     "mqtt_conack_flags_reserved",
                     "mqtt_conack_flags_sp", 
                     "mqtt_conflag_qos", 
                     "mqtt_conflag_reserved", 
                     "mqtt_conflag_retain", 
                     "mqtt_conflag_willflag", 
                     "mqtt_willtopic", 
                     "mqtt_willtopic_len", 
                     "mqtt_sub_qos", 
                     "mqtt_suback_qos", 
                     "mqtt_willmsg",
                     "mqtt_willmsg_len"]



binary_cols = ["mqtt_conack_val", 
                  "mqtt_conflag_cleansess", 
                  "mqtt_conflag_passwd", 
                  "mqtt_conflag_uname", 
                  "mqtt_dupflag", 
                  "mqtt_proto_len", 
                  "mqtt_protoname", 
                  "mqtt_qos", 
                  "mqtt_retain", 
                  "mqtt_ver"]


continuous_cols = ["tcp_flags_decimal",
                     'tcp_time_delta',
                     'tcp_len',
                     'mqtt_kalive',
                     'mqtt_len',
                     'mqtt_msgid',
                     'mqtt_msgtype',
                     'mqtt_conflags_decimal', 
                     "mqtt_hdrflags_decimal", ]

len(nominal_cols+corelated_cols_to_remove+binary_cols+continuous_cols)
# len(df_train_updated.columns)


33

In [110]:
#####Q1.1


class OutcomeCreater(Transformer): # this defines a transformer that creates the outcome column
    
    def __init__(self):
        super().__init__()
        

    def _transform(self, dataset):
        
        def converter(name):
            if name == 'legitimate':
                return 0
            elif name == 'dos':
                return 1      
            elif name == 'malformed':
                return 2     
            elif name == 'flood':
                return 3
            elif name == 'bruteforce':
                return 4   
            elif name == 'slowite':
                return 5  
            else:
                print("out of range")
                return None
            
        label_to_binary = udf(lambda name: converter(name))
        output_df = dataset.withColumn('label', label_to_binary(col('target')))
        output_df = output_df.drop("target")  
        output_df = output_df.withColumn('label', col('label').cast('int'))
        return output_df
    
class FeatureTypeCaster(Transformer): # this transformer will cast the columns as appropriate types  
    def __init__(self):
        super().__init__()

    def _transform(self, dataset):
        output_df = dataset
        for col_name in binary_cols + continuous_cols:
            output_df = output_df.withColumn(col_name,col(col_name).cast('float'))
        return output_df
    
class ColumnDropper(Transformer): # this transformer drops unnecessary columns
    def __init__(self, columns_to_drop = None):
        super().__init__()
        self.columns_to_drop=columns_to_drop
    def _transform(self, dataset):
        output_df = dataset
        for col_name in self.columns_to_drop:
            output_df = output_df.drop(col_name)
        return output_df
    
    
def get_preprocess_pipeline():

    # Stage where columns are casted as appropriate types
    stage_typecaster = FeatureTypeCaster()
    
    # Stage where nominal columns are transformed to index columns using StringIndexer
#     nominal_id_cols = [x+"_index" for x in nominal_cols]
#     nominal_onehot_cols = [x+"_encoded" for x in nominal_cols]
#     stage_nominal_indexer = StringIndexer(inputCols = nominal_cols, outputCols = nominal_id_cols )

    # Stage where the index columns are further transformed using OneHotEncoder
#     stage_nominal_onehot_encoder = OneHotEncoder(inputCols=nominal_id_cols, outputCols=nominal_onehot_cols)

    # Stage where all relevant features are assembled into a vector (and dropping a few)
    feature_cols = continuous_cols+binary_cols
    
#     drop_counter = 0
    
#     for col_name in corelated_cols_to_remove:
#         feature_cols.remove(col_name)
#         drop_counter += 1
        
#     print(drop_counter)

#     stage_vector_assembler = VectorAssembler(inputCols=feature_cols, outputCol="vectorized_features").setHandleInvalid("skip")
#     stage_vector_assembler = VectorAssembler(inputCols=feature_cols, outputCol="features").setHandleInvalid("skip")
#     stage_vector_assembler = VectorAssembler(inputCols=feature_cols, outputCol="features").setHandleInvalid("keep")

    # Stage where we scale the columns
    
#     stage_scaler = StandardScaler(inputCol= 'vectorized_features', outputCol= 'features')
    

    # Stage for creating the outcome column representing whether there is attack 
    stage_outcome = OutcomeCreater()

    # Removing all unnecessary columbs, only keeping the 'features' and 'outcome' columns
    stage_column_dropper = ColumnDropper(columns_to_drop = corelated_cols_to_remove+ ['vectorized_features', "dataset"])
    # Connect the columns into a pipeline
    pipeline = Pipeline(stages=[stage_typecaster,stage_outcome,stage_column_dropper])
#     pipeline = Pipeline(stages=[stage_typecaster,stage_nominal_indexer,stage_nominal_onehot_encoder, stage_typecaster])
    return pipeline 



In [118]:
df_train_updated.show(1, vertical = True)

-RECORD 0--------------------------------
 tcp_time_delta             | 0.998867   
 tcp_len                    | 10         
 mqtt_conack_flags          | 0          
 mqtt_conack_flags_reserved | 0.0        
 mqtt_conack_flags_sp       | 0.0        
 mqtt_conack_val            | 0.0        
 mqtt_conflag_cleansess     | 0.0        
 mqtt_conflag_passwd        | 0.0        
 mqtt_conflag_qos           | 0.0        
 mqtt_conflag_reserved      | 0.0        
 mqtt_conflag_retain        | 0.0        
 mqtt_conflag_uname         | 0.0        
 mqtt_conflag_willflag      | 0.0        
 mqtt_dupflag               | 0.0        
 mqtt_kalive                | 0.0        
 mqtt_len                   | 8.0        
 mqtt_msgid                 | 0.0        
 mqtt_msgtype               | 3.0        
 mqtt_proto_len             | 0.0        
 mqtt_protoname             | 0          
 mqtt_qos                   | 0.0        
 mqtt_retain                | 0.0        
 mqtt_sub_qos               | 0.0 

In [117]:
df_test_updated.show(1, vertical = True)

-RECORD 0-------------------------------
 tcp_time_delta             | 0.029854  
 tcp_len                    | 0         
 mqtt_conack_flags          | 0         
 mqtt_conack_flags_reserved | 0.0       
 mqtt_conack_flags_sp       | 0.0       
 mqtt_conack_val            | 0.0       
 mqtt_conflag_cleansess     | 0.0       
 mqtt_conflag_passwd        | 0.0       
 mqtt_conflag_qos           | 0.0       
 mqtt_conflag_reserved      | 0.0       
 mqtt_conflag_retain        | 0.0       
 mqtt_conflag_uname         | 0.0       
 mqtt_conflag_willflag      | 0.0       
 mqtt_dupflag               | 0.0       
 mqtt_kalive                | 0.0       
 mqtt_len                   | 0.0       
 mqtt_msgid                 | 0.0       
 mqtt_msgtype               | 0.0       
 mqtt_proto_len             | 0.0       
 mqtt_protoname             | 0         
 mqtt_qos                   | 0.0       
 mqtt_retain                | 0.0       
 mqtt_sub_qos               | 0.0       
 mqtt_suback_qos

In [111]:
preprocess_pipeline = get_preprocess_pipeline()
preprocess_pipeline_model = preprocess_pipeline.fit(df_train_updated)

df_train_final = preprocess_pipeline_model.transform(df_train_updated)
df_test_final = preprocess_pipeline_model.transform(df_test_updated)


df_train_pandas = df_train_final.toPandas()
df_test_pandas = df_test_final.toPandas()
# df_test_pandas.to_csv( "/Users/sz904/Desktop/df_test_tf.csv", index=False)
# df_train_pandas.to_csv( "/Users/sz904/Desktop/df_train_tf.csv", index=False)






In [120]:
df_test_pandas

,tcp_time_delta,tcp_len,mqtt_conack_val,mqtt_conflag_cleansess,mqtt_conflag_passwd,mqtt_conflag_uname,mqtt_dupflag,mqtt_kalive,mqtt_len,mqtt_msgid,mqtt_msgtype,mqtt_proto_len,mqtt_protoname,mqtt_qos,mqtt_retain,mqtt_ver,tcp_flags_decimal,mqtt_conflags_decimal,mqtt_hdrflags_decimal,label
0,0.029854,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,0.0,0.0,2
1,0.000001,51.0,0.0,0.0,0.0,0.0,1.0,0.0,171.0,5754.0,3.0,0.0,0.0,1.0,0.0,0.0,24.0,0.0,58.0,1
2,0.999820,13.0,0.0,0.0,0.0,0.0,0.0,0.0,11.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,48.0,0
3,0.007949,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,2
4,0.000121,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99285,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,4
99286,0.000120,14.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,48.0,0
99287,0.001063,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,4
99288,0.000026,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,1


In [119]:
df_train_pandas

,tcp_time_delta,tcp_len,mqtt_conack_val,mqtt_conflag_cleansess,mqtt_conflag_passwd,mqtt_conflag_uname,mqtt_dupflag,mqtt_kalive,mqtt_len,mqtt_msgid,mqtt_msgtype,mqtt_proto_len,mqtt_protoname,mqtt_qos,mqtt_retain,mqtt_ver,tcp_flags_decimal,mqtt_conflags_decimal,mqtt_hdrflags_decimal,label
0,0.998867,10.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,48.0,0
1,0.000067,1460.0,0.0,0.0,0.0,0.0,0.0,0.0,169.0,2714.0,3.0,0.0,0.0,1.0,0.0,0.0,16.0,0.0,50.0,1
2,0.000058,1460.0,0.0,0.0,0.0,0.0,0.0,0.0,163.0,1548.0,3.0,0.0,0.0,1.0,0.0,0.0,16.0,0.0,50.0,1
3,0.000227,10.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,48.0,0
4,0.000236,16.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2800.0,4.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,64.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231641,0.000003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0
231642,0.000003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0
231643,0.000010,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0
231644,0.000001,12.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,6399.0,4.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,64.0,1


In [85]:
df_train_final.printSchema()

root
 |-- tcp_time_delta: float (nullable = true)
 |-- tcp_len: float (nullable = true)
 |-- mqtt_conack_val: float (nullable = true)
 |-- mqtt_conflag_cleansess: float (nullable = true)
 |-- mqtt_conflag_passwd: float (nullable = true)
 |-- mqtt_conflag_uname: float (nullable = true)
 |-- mqtt_dupflag: float (nullable = true)
 |-- mqtt_kalive: float (nullable = true)
 |-- mqtt_len: float (nullable = true)
 |-- mqtt_msgid: float (nullable = true)
 |-- mqtt_msgtype: float (nullable = true)
 |-- mqtt_proto_len: float (nullable = true)
 |-- mqtt_protoname: float (nullable = true)
 |-- mqtt_qos: float (nullable = true)
 |-- mqtt_retain: float (nullable = true)
 |-- mqtt_ver: float (nullable = true)
 |-- tcp_flags_decimal: float (nullable = true)
 |-- mqtt_conflags_decimal: float (nullable = true)
 |-- mqtt_hdrflags_decimal: float (nullable = true)
 |-- label: integer (nullable = true)



In [86]:
# from sklearn.metrics import confusion_matrix
# from pyspark.ml.classification import RandomForestClassifier
# from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# from pyspark.ml.classification import DecisionTreeClassifier


In [87]:
# rf = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'label', maxBins = 8)
# rf_model = rf.fit(df_train_final)

In [112]:
from tensorflow import keras
import tensorflow as tf  # now import the tensorflow module
print(tf.__version__)  # make sure the version is 2.x
# from pyspark.sql.functions import udf
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

2.10.0


In [121]:
# df_train_tf = pd.read_csv("/Users/sz904/Desktop/df_train_tf.csv")
# df_test_tf = pd.read_csv("/Users/sz904/Desktop/df_test_tf.csv")

df_train_tf = df_train_pandas
df_test_tf = df_test_pandas

x_valid, x_test, y_valid, y_test = train_test_split( df_test_tf.iloc[:, 0:-1], df_test_tf.iloc[:, -1:], test_size=0.33, random_state=42)

df_train_tf = df_train_tf.iloc[:100000, :]

x_train = df_train_tf.iloc[:, 0:-1]
y_train = df_train_tf.iloc[:, -1:]
y_train=np.ravel(y_train)



# x_valid = df_valid.iloc[:, 0:-1]
# y_valid = df_valid.iloc[:, -1:]
# y_valid=np.ravel(y_valid)

# x_test =  df_test.iloc[:, 0:-1]
# y_test = df_test.iloc[:, -1:]
# y_test=np.ravel(y_test)

scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
x_valid = scaler.transform(x_valid)

# x_train = tf.constant(np.array(df_train_pandas['features'].values.tolist()))
# y_train = tf.constant(np.array(df_train_pandas['label'].values.tolist()))

# x_validate = tf.constant(np.array(df_valid_pandas['features'].values.tolist()))
# y_validate = tf.constant(np.array(df_valid_pandas['label'].values.tolist()))


# x_test = tf.constant(np.array(df_test_pandas['features'].values.tolist()))
# y_test = tf.constant(np.array(df_test_pandas['label'].values.tolist()))


In [122]:
x_train

array([[ 0.2544472 , -0.1518746 , -0.06700799, ...,  0.82465214,
        -0.06663252,  0.4842833 ],
       [-0.09625363,  1.4371164 , -0.06700799, ..., -0.95063066,
        -0.06663252,  0.55086243],
       [-0.09625679,  1.4371164 , -0.06700799, ..., -0.95063066,
        -0.06663252,  0.55086243],
       ...,
       [-0.09624975,  1.4371164 , -0.06700799, ..., -0.95063066,
        -0.06663252,  0.55086243],
       [-0.09627469, -0.14858703, -0.06700799, ...,  0.82465214,
        -0.06663252,  0.4842833 ],
       [-0.09620552, -0.15406631, -0.06700799, ...,  0.82465214,
        -0.06663252,  1.0169164 ]], dtype=float32)

In [125]:
x_valid

array([[-0.09623852, -0.14858703, -0.06700799, ...,  0.82465214,
        -0.06663252,  0.4842833 ],
       [-0.09627399, -0.16283315, -0.06700799, ..., -0.95063066,
        -0.06663252, -1.1136159 ],
       [-0.09627539, -0.16283315, -0.06700799, ..., -0.95063066,
        -0.06663252, -1.1136159 ],
       ...,
       [ 0.25482818, -0.1518746 , -0.06700799, ...,  0.82465214,
        -0.06663252,  0.4842833 ],
       [-0.09624343,  1.4371164 , -0.06700799, ..., -0.95063066,
        -0.06663252,  0.55086243],
       [ 0.25478113, -0.14858703, -0.06700799, ...,  0.82465214,
        -0.06663252,  0.4842833 ]], dtype=float32)

In [128]:
x_test

array([[ 0.2548243 , -0.14749117, -0.06700799, ...,  0.82465214,
        -0.06663252,  0.4842833 ],
       [-0.09622483, -0.14749117, -0.06700799, ...,  0.82465214,
        -0.06663252,  0.4842833 ],
       [-0.09627715, -0.07735639, -0.06700799, ...,  0.82465214,
        -0.06663252,  0.81717896],
       ...,
       [-0.09627679, -0.0894108 , -0.06700799, ...,  0.82465214,
        -0.06663252,  0.81717896],
       [-0.09627715, -0.16283315, -0.06700799, ..., -0.95063066,
        -0.06663252, -1.1136159 ],
       [-0.09627469, -0.16283315, -0.06700799, ..., -0.95063066,
        -0.06663252, -1.1136159 ]], dtype=float32)

In [123]:
# import tensorflow_decision_forests as tfdf
# import os
# os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [124]:
# model_1 = tfdf.keras.RandomForestModel(verbose=2)
# model_1.fit(x=df_train_tf)


In [126]:
model = keras.Sequential( [keras.layers.Dense(10,activation='relu'),
                           keras.layers.Dense(10,activation='tanh'),

                           keras.layers.Dense(10,activation='tanh') ,
                           keras.layers.BatchNormalization(epsilon=1e-06,  momentum=0.9, weights=None),
                           keras.layers.Dense(6)] )


In [127]:
log_dir_Q1_2 = "/Users/sz904/Desktop/"

model.compile(optimizer = keras.optimizers.SGD(),
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = [keras.metrics.SparseCategoricalAccuracy(name="Accuracy_epochs")])

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir_Q1_2, histogram_freq=1)

model.fit(x=x_train, y=y_train, 
          epochs=15, verbose = 2,
          validation_data=(x_valid, y_valid), 
          callbacks=[tensorboard_callback])

Epoch 1/15


2022-11-26 22:03:45.351656: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-11-26 22:03:58.996122: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3125/3125 - 22s - loss: 1.1090 - Accuracy_epochs: 0.4952 - val_loss: 1.0759 - val_Accuracy_epochs: 0.5015 - 22s/epoch - 7ms/step
Epoch 2/15
3125/3125 - 21s - loss: 1.0749 - Accuracy_epochs: 0.4996 - val_loss: 1.0745 - val_Accuracy_epochs: 0.5015 - 21s/epoch - 7ms/step
Epoch 3/15


KeyboardInterrupt: 

In [13]:
to_array = udf(lambda v: v.toArray().tolist(), ArrayType(FloatType()))

# df_train, df_abandon = df_train_final.randomSplit([0.01,0.99])
# df_test_temp, df_abandon1 = df_test_final.randomSplit([0.2,0.8])
df_valid,df_test = df_test_temp.randomSplit([0.5,0.5])

df_train_pandas = df_train.withColumn('features', to_array('features')).toPandas()
df_valid_pandas = df_valid.withColumn('features', to_array('features')).toPandas()
df_test_pandas = df_test.withColumn('features', to_array('features')).toPandas()


22/11/26 16:21:45 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
22/11/26 16:21:45 WARN DAGScheduler: Broadcasting large task binary with size 16.3 MiB


22/11/26 16:21:52 ERROR Executor: Exception in task 1.0 in stage 7.0 (TID 29)
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.util.Arrays.copyOf(Arrays.java:3481)
	at java.base/java.util.ArrayList.trimToSize(ArrayList.java:204)
	at net.razorvine.pickle.Unpickler.load_appends(Unpickler.java:727)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:232)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:109)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:122)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.$anonfun$evaluate$6(BatchEvalPythonExec.scala:94)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec$$Lambda$4389/0x000000080207b4d8.apply(Unknown Source)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:492)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.

Py4JJavaError: An error occurred while calling o661.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 7.0 failed 1 times, most recent failure: Lost task 1.0 in stage 7.0 (TID 29) (stevens-mbp-4 executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.base/java.util.Arrays.copyOf(Arrays.java:3481)
	at java.base/java.util.ArrayList.trimToSize(ArrayList.java:204)
	at net.razorvine.pickle.Unpickler.load_appends(Unpickler.java:727)
	at net.razorvine.pickle.Unpickler.dispatch(Unpickler.java:232)
	at net.razorvine.pickle.Unpickler.load(Unpickler.java:109)
	at net.razorvine.pickle.Unpickler.loads(Unpickler.java:122)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.$anonfun$evaluate$6(BatchEvalPythonExec.scala:94)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec$$Lambda$4389/0x000000080207b4d8.apply(Unknown Source)
	at scala.collection.Iterator$$anon$11.nextCur(Iterator.scala:486)
	at scala.collection.It

22/11/26 16:21:56 WARN PythonUDFRunner: Incomplete task 3.0 in stage 7 (TID 31) interrupted: Attempting to kill Python Worker
22/11/26 16:21:56 WARN PythonUDFRunner: Incomplete task 6.0 in stage 7 (TID 34) interrupted: Attempting to kill Python Worker
22/11/26 16:21:56 WARN PythonUDFRunner: Incomplete task 2.0 in stage 7 (TID 30) interrupted: Attempting to kill Python Worker
22/11/26 16:21:56 WARN PythonUDFRunner: Incomplete task 3.0 in stage 7 (TID 31) interrupted: Attempting to kill Python Worker
22/11/26 16:21:56 WARN PythonUDFRunner: Incomplete task 4.0 in stage 7 (TID 32) interrupted: Attempting to kill Python Worker
22/11/26 16:21:56 WARN PythonUDFRunner: Incomplete task 7.0 in stage 7 (TID 35) interrupted: Attempting to kill Python Worker
22/11/26 16:21:56 WARN PythonUDFRunner: Incomplete task 5.0 in stage 7 (TID 33) interrupted: Attempting to kill Python Worker
22/11/26 16:21:56 WARN PythonUDFRunner: Incomplete task 4.0 in stage 7 (TID 32) interrupted: Attempting to kill Python

----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 50843)
Traceback (most recent call last):
  File "/Users/sz904/miniforge3/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/sz904/miniforge3/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/Users/sz904/miniforge3/lib/python3.10/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/sz904/miniforge3/lib/python3.10/socketserver.py", line 747, in __init__
    self.handle()
  File "/Users/sz904/miniforge3/lib/python3.10/site-packages/pyspark/accumulators.py", line 281, in handle
    poll(accum_updates)
  File "/Users/sz904/miniforge3/lib/python3.10/site-packages/pyspark/accumulators.py", line 253, in poll
    if func():
  File "/Users/sz904/miniforge3/lib/python3.10